<a href="https://colab.research.google.com/github/Monisha2604/Final_Year_Project/blob/master/ANN_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install keras

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
!tar xf spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
# spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
pip install elephas

     |████████████████████████████████| 337kB 14.1MB/s 
     |████████████████████████████████| 2.9MB 43.7MB/s 
     |████████████████████████████████| 212.4MB 63kB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 204kB 47.9MB/s 
  Created wheel for elephas: filename=elephas-2.1.0-cp37-none-any.whl size=27611 sha256=851198d2dc834807f15d82d7313fc06fe49f549aabe40b84206264dbe1291f9a
  Stored in directory: /root/.cache/pip/wheels/b4/8e/f7/afeaa15a424e0df01ff445dea2ac4cfddda282329494bbb027
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=d9a9acd0e28582d71e48cd32cacca1b21f9721112c7ad07a5acab454c60213ec
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built elephas pyspark
ERROR: tensorflow 2.5.0 has requirement h5py~=3.1.0, but you'll have h5py 2.10.0 which is incompatible.
  Found existing installation: h5py 3.1.0
    Uninstalli

In [ ]:
from elephas.ml_model import ElephasEstimator
from elephas.ml.adapter import to_data_frame
 
 
from pyspark import SparkContext, SparkConf
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.ml import Pipeline

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from pyspark.sql import SQLContext
from pyspark.ml.linalg import Vectors
import numpy as np
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/MyTable_13_snigdha.csv")

In [ ]:
df.columns

Index(['objid', 'ra', 'dec', 'psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i',
       'psfMag_z', 'dered_u', 'dered_g', 'dered_r', 'dered_i', 'dered_z',
       'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i', 'fiberMag_z',
       'petroR50_r', 'petroR50_z', 'petroR90_r', 'petroR90_z', 'r', 'i', 'z',
       'redshift'],
      dtype='object')

In [ ]:
df=df.drop(['objid', 'ra', 'dec'],axis=1)

In [ ]:
col=df.columns
len(col)

23

In [ ]:
df[df["dered_u"]==-9999.0]=np.nan
df[df["dered_g"]==-9999.0]=np.nan
df[df["dered_r"]==-9999.0]=np.nan
df[df["dered_i"]==-9999.0]=np.nan
df[df["dered_z"]==-9999.0]=np.nan

df[df["fiberMag_u"]==-9999]=np.nan
df[df["fiberMag_g"]==-9999]=np.nan
df[df["fiberMag_r"]==-9999]=np.nan
df[df["fiberMag_i"]==-9999]=np.nan
df[df["fiberMag_z"]==-9999]=np.nan

df[df["r"]==-9999]=np.nan
df[df["i"]==-9999]=np.nan
df[df["z"]==-9999]=np.nan
df[df["petroR50_r"]==-9999]=np.nan
df[df["petroR90_r"]==-9999]=np.nan

df[df["psfMag_g"]==-9999.0]=np.nan
df[df["psfMag_r"]==-9999.0]=np.nan
df[df["psfMag_i"]==-9999.0]=np.nan
df[df["psfMag_z"]==-9999.0]=np.nan

In [ ]:
df=df.dropna()

In [ ]:
df.isnull().sum()

psfMag_u      0
psfMag_g      0
psfMag_r      0
psfMag_i      0
psfMag_z      0
dered_u       0
dered_g       0
dered_r       0
dered_i       0
dered_z       0
fiberMag_u    0
fiberMag_g    0
fiberMag_r    0
fiberMag_i    0
fiberMag_z    0
petroR50_r    0
petroR50_z    0
petroR90_r    0
petroR90_z    0
r             0
i             0
z             0
redshift      0
dtype: int64

In [ ]:
df.to_csv("redshift.csv")

In [ ]:
conf=SparkConf().setAppName('MLP').setMaster('local[*]')
sc=SparkContext(conf=conf)
sql_context=SQLContext(sc)

In [ ]:
data=sql_context.read.csv("/content/redshift.csv",header=True,inferSchema=True)

In [ ]:
data=data.sample(False,0.4,1234)

In [ ]:
data.count()

996495

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler=VectorAssembler(
    inputCols=['psfMag_u',
               'psfMag_g',
               'psfMag_r',
               'psfMag_i',
               'psfMag_z',
               'dered_u', 
              'dered_g', 
               'dered_r', 
               'dered_i', 
               'dered_z', 
               'fiberMag_u', 
               'fiberMag_g', 
               'fiberMag_r', 
              'fiberMag_i', 
               'fiberMag_z', 
               'petroR50_r',
               'petroR50_z',
                'petroR90_r',
                'petroR90_z',
               'r', 
               'i', 
               'z'],
    outputCol="features"
)

In [ ]:
data=assembler.transform(data)

In [ ]:
output=data.select("features","redshift")

In [ ]:
(train_df,test_df)=output.randomSplit([0.8,0.2])

In [ ]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense,Dropout,BatchNormalization
from tensorflow.keras import regularizers

In [ ]:
from tensorflow.keras import optimizers

In [ ]:
input_dim = len(train_df.select("features").first()[0])

In [ ]:
model = Sequential()
 
'''
model = Sequential()
model.add(Dense(12, input_dim=25, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))'''
 
model.add(Dense(256, input_shape=(input_dim,))) # shape[1] will be number of columns
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))

#kernel_regularizer=regularizers.l2(0.001)

model.add(Dense(256))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))
 
model.add(Dense(256))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))
 
model.add(Dense(1))
model.add(Activation('linear'))

In [ ]:
opt = optimizers.Adam(learning_rate=0.0001)
sgd_conf = optimizers.serialize(opt)
# no Adamax
#no Adam
#Adadelta
#no RMSprop
  #opt = optimizers.RMSprop(lr=0.01)
  #sgd_conf = optimizers.serialize(opt)

In [ ]:
estimator = ElephasEstimator()
estimator.set_keras_model_config(model.to_yaml())
estimator.set_optimizer_config(sgd_conf)
estimator.set_mode("synchronous")
estimator.set_loss("mse")
estimator.set_metrics(['mse'])
estimator.set_epochs(10)
estimator.set_batch_size(500)
estimator.set_validation_split(0.10)
estimator.set_categorical_labels(False)
estimator.set_loss("mse")
estimator.set_metrics(["acc"])

estimator.setFeaturesCol("features")
estimator.setLabelCol("label")

ElephasEstimator_8d7bddc18275

In [ ]:

estimator = ElephasEstimator()
estimator.set_keras_model_config(model.to_yaml())
estimator.set_optimizer_config(sgd_conf)
estimator.setFeaturesCol("features")
estimator.setLabelCol("label")
estimator.set_mode("synchronous")
estimator.set_loss("mae")
estimator.set_metrics(['mae'])
estimator.set_epochs(100)
estimator.set_batch_size(256)
estimator.set_validation_split(0.01)
estimator.set_categorical_labels(False)

ElephasEstimator_f066e0a23840

In [ ]:
from pyspark.ml import Pipeline
 
pipeline = Pipeline(stages=[estimator])

In [ ]:
train_df=train_df.withColumnRenamed("redshift","label")

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics
import timeit 
start = timeit.default_timer()
fitted_pipeline = pipeline.fit(train_df) # Fit model to data
stop = timeit.default_timer()
execution_time = (stop - start)/60
print("Program Executed in minutes "+str(execution_time))

>>> Fit model
>>> Synchronous training complete.
Program Executed in minutes 36.71486127196665


In [ ]:
test_df.show(2)

+--------------------+----------+
|            features|     label|
+--------------------+----------+
|[15.2698299999999...|       0.0|
|[15.72344,15.3647...|0.04702275|
+--------------------+----------+
only showing top 2 rows



In [ ]:
prediction = fitted_pipeline.transform(test_df) # <-- The same code evaluates test data.

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
evaluator = RegressionEvaluator(labelCol="redshift", predictionCol="prediction", metricName="r2")
rmse = evaluator.evaluate(prediction)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)


Root Mean Squared Error (RMSE) on test data = -0.627669


In [ ]:
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel